# Crowd Funding Study Data Cleaning

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


In [2]:
#pd.set_option('display.max_columns', None) #display all columns in jupyter
df= pd.read_csv('scrapedCrowdOfferingsData.csv',index_col=None)

C:\Users\apsin\AppData\Local\Temp\ipykernel_21004\1480657344.py:2: DtypeWarning: Columns (46) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('scrapedCrowdOfferingsData.csv',index_col=None)


In [3]:
df.dtypes

accessionNumber                      object
compensationAmount                   object
financialInterest                    object
securityOfferedType                  object
securityOfferedOtherDesc             object
noOfSecurityOffered                 float64
price                               float64
priceDeterminationMethod             object
targetOfferingAmount                float64
oversubscriptionAccepted             object
oversubscriptionAllocationType       object
oversubscriptionDesc                 object
maximumOfferingAmount               float64
campaignDeadlineDate                 object
currentEmployees                    float64
totalAssetMostRecentFiscalYear      float64
totalAssetPriorFiscalYear           float64
cashEqMostRecentFiscalYear          float64
cashEqPriorFiscalYear               float64
accountsReceivedRecentFiscalYear    float64
accountsReceivedPriorFiscalYear     float64
shortTermDebtMRecentFiscalYear      float64
shortTermDebtPriorFiscalYear    

A lot of the object column types e.g. compensationAmount, financialInterest, intuitively should be numerical columns so let's look into object columns with more detail

In [9]:
df['dateIncorporation']= pd.to_datetime(df['dateIncorporation']).dt.date #converting to datetime type
df.select_dtypes(['object']).head(5)

,accessionNumber,compensationAmount,financialInterest,securityOfferedType,securityOfferedOtherDesc,priceDeterminationMethod,oversubscriptionAccepted,oversubscriptionAllocationType,oversubscriptionDesc,campaignDeadlineDate,...,jurisdictionOrganization,dateIncorporation,city,stateOrCountry,zipCode,issuerWebsite,IntermediaryName,isCoIssuer,submissionType,fileNumber
0,0001669191-23-000658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,DE,2016-08-05,NEW YORK,NY,10004,https://rialtomarkets.com/,NaN,N,C-AR,020-29259
1,0001781955-23-000008,3.5% of final raise amount paid to PicMii Crow...,None.,Common Stock,NaN,At issuer's discretion.,Y,"First-come, first-served basis",NaN,2023-08-01,...,DE,2019-04-16,HACKETTSTOWN,NJ,07840,www.domeaudioinc.com,PicMii Crowdfunding LLC,N,C/A,020-28102
2,0001669191-23-000655,Up to 4.9% of amount raised for a successful o...,None.,Common Stock,NaN,The price of the Securities was determined sol...,Y,"First-come, first-served basis",NaN,2023-08-31,...,DE,2021-11-22,Boston,MA,02110,https://custcorp.com/,NetCapital Funding Portal Inc.,N,C/A,020-29958
3,0001669191-23-000656,Up to 4.9% of amount raised for a successful o...,None.,Common Stock,NaN,The price of the Securities was determined sol...,Y,"First-come, first-served basis",NaN,2023-09-29,...,DE,2007-09-10,Hollister,CA,95023,https://www.lightningmotorcycle.com,NetCapital Funding Portal Inc.,N,C/A,020-32002
4,0001670254-23-000688,4.0% of the offering amount upon a successful ...,No,Other,Simple Agreement for Future Equity (SAFE),Pro-rated portion of the total principal value...,Y,Other,As determined by the issuer,2024-04-29,...,DE,2012-04-05,Stamford,CT,06901,https://www.clearingbid.com/,Wefunder Portal LLC,Y,C/A,020-32489


- **Dropping columns**
    - Seems like compensation amount, priceDeterminationMethod is in text form, and there is no clear uniform consistency to extract features from.
    - The financial interest is also in text form, a derived boolean column in the form of yes/no to include this feature in model if required later. 

In [10]:

df.drop(columns=['compensationAmount','financialInterest'],axis=1, inplace=True)

In [13]:
df.select_dtypes(['object'])[df['progressUpdate'].notnull()].head(3)

,accessionNumber,securityOfferedType,securityOfferedOtherDesc,priceDeterminationMethod,oversubscriptionAccepted,oversubscriptionAllocationType,oversubscriptionDesc,campaignDeadlineDate,progressUpdate,natureOfAmendment,...,jurisdictionOrganization,dateIncorporation,city,stateOrCountry,zipCode,issuerWebsite,IntermediaryName,isCoIssuer,submissionType,fileNumber
7,0001747584-23-000032,Debt,NaN,NaN,Y,Other,"See ""Allocations of Investments"" section in th...",2023-05-18,"The issuer closed on $250,000 of debt notes.",NaN,...,CT,2021-10-15,HARTFORD,MA,06106,invest.ctgreenbank.com,"Raise Green, Inc.",N,C-U,020-32179
27,0001747584-23-000031,Other,Simple Agreement for Future Equity,NaN,Y,"First-come, first-served basis",NaN,2023-05-25,"The issuer closed on $32,000 of SAFEs. The fun...",NaN,...,DE,2019-06-13,AUSTIN,TX,78702,https://www.appliedbioplastics.com/,"Raise Green, Inc.",N,C-U,020-31471
43,0001670254-23-000676,Other,Revenue Participation Agreement,Pro-rated portion of the total principal value...,Y,Other,As determined by the issuer,2024-04-29,"The final number is $231,205.00 in investments...",NaN,...,NY,2022-07-11,Brooklyn,NY,11215,http://www.threeyearsgone.com,Wefunder Portal LLC,N,C-U,020-31836



**Progress update is in text form but seems like this is where the data for raisedOfferingAmount would come for after some processing so will not drop it**

In [15]:
cardinality= df.select_dtypes(['object']).nunique()
print(cardinality)


accessionNumber                   23474
securityOfferedType                   4
securityOfferedOtherDesc            604
priceDeterminationMethod           1307
oversubscriptionAccepted              2
oversubscriptionAllocationType        3
oversubscriptionDesc                162
campaignDeadlineDate               2092
progressUpdate                     3382
natureOfAmendment                  5043
issuerName                         7420
legalStatusForm                       6
legalStatusOtherDesc                 25
jurisdictionOrganization             53
dateIncorporation                  2916
city                               2674
stateOrCountry                       70
zipCode                            3442
issuerWebsite                      7682
IntermediaryName                    209
isCoIssuer                            2
submissionType                       11
fileNumber                         7319
dtype: int64


In [16]:
(df.isna().sum()/len(df))*100 #percentage of nan values in each column

accessionNumber                      0.000000
securityOfferedType                 20.456675
securityOfferedOtherDesc            65.148675
noOfSecurityOffered                 36.014314
price                               24.631507
priceDeterminationMethod            54.319673
targetOfferingAmount                20.456675
oversubscriptionAccepted            20.456675
oversubscriptionAllocationType      21.415183
oversubscriptionDesc                53.527307
maximumOfferingAmount               21.415183
campaignDeadlineDate                20.456675
currentEmployees                     6.500809
totalAssetMostRecentFiscalYear       6.500809
totalAssetPriorFiscalYear            6.500809
cashEqMostRecentFiscalYear           6.500809
cashEqPriorFiscalYear                6.500809
accountsReceivedRecentFiscalYear     6.500809
accountsReceivedPriorFiscalYear      6.500809
shortTermDebtMRecentFiscalYear       6.500809
shortTermDebtPriorFiscalYear         6.500809
longTermDebtRecentFiscalYear      

### Interesting observations
- each of the financials have 6.5% of the total values are Nan indicated a system trend
- No missing values in isAmendment, submissionType, so these can be used as filters
- issuerCIK and issuerName are both have same percentage of missing values, indicating that in a record both these fields are likely missing together
- Some categorical columns can be dropped if cardinality or if the no of missing values are high: ['compensationAmount', 'financialInterest',
       'securityOfferedOtherDesc', 'priceDeterminationMethod', 'oversubscriptionDesc', 'natureOfAmendment', 'legalStatusOtherDesc','jurisdictionOrganization',
       'city','zipCode', 'issuerWebsite']
- IssuerCIK is unique to IssuerName and as such doesn't provide any additional information so can be dropped as well.

In [19]:
columns_to_drop= ['securityOfferedOtherDesc', 'priceDeterminationMethod', 'oversubscriptionDesc', 'natureOfAmendment', 'legalStatusOtherDesc'
,'jurisdictionOrganization','city','zipCode', 'issuerWebsite', 'issuerCIK']
df.drop(columns=columns_to_drop, axis=1,inplace=True)

## Submission Type Context

Description of the various categories in submissionType is available at https://www.sec.gov/dera/data/crowdfund.pdf

- C (Offering Statement): If the crowdfunding campaign was launched, but no funds were raised, the initial offering statement would have been filed. However, this may not necessarily be updated if no funds are raised.
- C-W (Offering Statement Withdrawal): If the issuer decides to cancel the crowdfunding campaign before raising any funds, they may file a withdrawal of the offering statement (C-W).
- C-U (Progress Update): Provides progress updates of the campaign.
- C-U-W (Progress Update Withdrawal): If the issuer initially filed a progress update (C-U) but later decides to withdraw it, they can file a withdrawal (C-U-W).
- C/A (Amendment to Offering Statement): If the issuer initially filed an offering statement (C) and then decided to make material changes or updates related to the campaign, they may file an amendment (C/A). However, this is less likely in the case of a failed campaign.
- C/A-W (Amendment to Offering Statement Withdrawal):If an issuer submitted an amendment to the offering statement (C/A) but later decided to withdraw it, they could file a withdrawal (C/A-W).
- C-AR (Annual Report): An annual report (C-AR) is typically filed by issuers who have successfully raised funds and continue to operate. It's less relevant if no funds were raised.
- C-AR-W (Annual Report Withdrawal): If an issuer filed an annual report (C-AR) but later decided to withdraw it, they could file a withdrawal (C-AR-W).
- C-AR/A (Amendment to Annual Report): An issuer may file an amendment (C-AR/A) to the annual report if there are material changes or updates to the previously submitted annual report. This is less likely in the case of a failed campaign.
- C-AR/A-W (Amendment to Annual Report Withdrawal): If an issuer submitted an amendment to the annual report (C-AR/A) but later decided to withdraw it, they could file a withdrawal (C-AR/A-W).
- C-TR (Termination of Reporting): If an issuer decides to cease ongoing reporting obligations to the SEC, they may file a termination of reporting (C-TR). This could apply if the campaign failed, and they no longer wish to provide updates.
- C-TR-W (Termination of Reporting Withdrawal): If an issuer initially filed a termination of reporting (C-TR) but later decided to withdraw it, they could file a withdrawal (C-TR-W).

In [ ]:
df['filingDate']= pd.to_datetime(df['filingDate']).dt.date
#df.drop(columns= ['issuerCIK'], inplace=True)
df['isAmendment']= df['isAmendment'].astype('bool')
num_cols= df.select_dtypes(include=['number']).columns.tolist()

#print(num_cols)
grouped =  df[~df['submissionType'].str.contains('W')].groupby(['fileNumber'])[num_cols].agg(lambda x: (x.max() - x.min()) / x.max() * 100)


In [66]:
df2=pd.DataFrame()
df2['countFilings']= df.groupby(['fileNumber'])['submissionType'].count() #count no of total filings
df2['countAmendments']= df[df['submissionType']== 'C/A'].groupby(['fileNumber'])['submissionType'].count() #count number of ammendments 
df2['countAmendments']= df2['countAmendments'].fillna(0) #fill null values from count as zero
df2['countProgressUpdates']= df[df['submissionType']== 'C-U'].groupby(['fileNumber'])['submissionType'].count() #count number of ammendments 
df2['countProgressUpdates']= df2['countProgressUpdates'].fillna(0) #fill null values from count as zero


In [71]:
def count_conditional(column):
    return (column == 0).sum()

# Apply the function to each column
counts_info= df2.apply(count_conditional)
counts_info/len(df2)*100

countFilings             0.000000
countAmendments         50.922257
countProgressUpdates    57.781118
dtype: float64